In [12]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
import requests

g = Graph()

DBO = Namespace("http://dbpedia.org/ontology/")
DBR = Namespace("http://dbpedia.org/resource/")
g.bind("dbo", DBO)
g.bind("dbr", DBR)

query = """
SELECT DISTINCT ?city ?cityName ?population ?monument ?monumentName
WHERE {
 VALUES ?europeanCountry { 
   dbr:France dbr:Germany dbr:Italy dbr:Spain dbr:United_Kingdom 
   dbr:Poland dbr:Romania dbr:Netherlands dbr:Belgium dbr:Greece
   dbr:Czech_Republic dbr:Portugal dbr:Sweden dbr:Hungary dbr:Austria 
 }
 ?city a dbo:City ;
       rdfs:label ?cityName ;
       dbo:country ?europeanCountry .
 OPTIONAL { ?city dbo:populationTotal ?population }
 FILTER(LANG(?cityName) = "en")
 OPTIONAL {
   ?monument a dbo:Monument ;
            dbo:location ?city ;
            rdfs:label ?monumentName .
   FILTER(LANG(?monumentName) = "en")
 }
}
"""

headers = {
    'Accept': 'application/sparql-results+json',
    'User-Agent': 'Mozilla/5.0'
}
endpoint = "https://dbpedia.org/sparql"
response = requests.get(endpoint, params={'query': query, 'format': 'json'}, headers=headers)

if response.status_code == 200:
    data = response.json()
    for result in data['results']['bindings']:
        city_uri = URIRef(result['city']['value'])
        g.add((city_uri, RDF.type, DBO.City))
        g.add((city_uri, RDFS.label, Literal(result['cityName']['value'], lang='en')))
        
        if 'population' in result:
            g.add((city_uri, DBO.populationTotal, Literal(int(result['population']['value']))))
        
        if 'monument' in result:
            monument_uri = URIRef(result['monument']['value'])
            g.add((monument_uri, RDF.type, DBO.Monument))
            g.add((monument_uri, DBO.location, city_uri))
            g.add((monument_uri, RDFS.label, Literal(result['monumentName']['value'], lang='en')))

    print(f"Number of triples: {len(g)}")
    g.serialize("dbpedia_cities.ttl", format="turtle")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Number of triples: 3883
